# PISA 2022 analysis

Some basic analysis on school student performance in PISA tests across OECD countries. in particular we examine
- the effect of GDP per capita
- Culture
- Gender
- etc.

In [1]:
import pandas as pd

## Loading data
(data source: https://www.oecd-ilibrary.org/education/pisa-2022-results-volume-i_53f23881-en )